In [1]:
import os
import tools
import numpy as np
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.22.4 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 pynndescent==0.5.2


In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
bdata = sc.read_h5ad('h5ad/df.fil3_gex_bcells_no_CLL_vdj_230622.h5ad')
bdata

AnnData object with n_obs × n_vars = 68683 × 330
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'RNA_readsConfidentlyOnTranscriptome', 'RNA_readsConfidentlyOnGenome', 'RNA_readsOnGenome', 'RNA_Q30readsBarcode', 'RNA_Q30readsRead', 'RNA_Q30readsUmi', 'rna_snn_res.4', 'rna_snn_res.0.5', 'rna_snn_res.0.1', 'harmony_snn_res.4', 'yoshida_broad_predLabel_majorVote', 'harmony_snn_res.0.5', 'harmony_snn_res.0.1', 'cell_type_compartment

In [4]:
adata = sc.read_h5ad('h5ad/df.fil3_gex_kt_230622.h5ad')
adata = adata[adata.obs_names.isin(bdata.obs_names)]

In [5]:
# before i try to do a complete swap of the obs
assert all(adata.obs.index == bdata.obs.index)

In [6]:
# transfer obs and obsm
adata.obs = bdata.obs.copy()
adata.obsm = bdata.obsm.copy()
# trim the columns
adata.obs = adata.obs[['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sample_date', 'temp_WCC', 'temp_neut', 'temp_mono', 'temp_lymph', 'temp_CRP', 'temp_ddimer', 'temp_ferritin', 'temp_trop', 'WHO_temp_severity', 'individual_id', 'celltype_B_corrected', 'celltype_B_corrected_IFN', 'corrected_ethnicity', 'PC1_nonafricanVsAfrican', 'PC2_asianVsEuropean']]
adata

AnnData object with n_obs × n_vars = 68683 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [7]:
# remove doublets
adata = adata[(adata.obs['merged_souporcell_status'] == 'singlet') & (adata.obs['celltype_B_corrected'] != 'doublet')].copy()
# remove the unannotated samples
adata = adata[(adata.obs['sample_id'] != 'NA') & (~adata.obs['patient_id'].isin(['NULL', 'unresolved'])) & (adata.obs['case_control'] != 'NA')].copy()
# remove patient C141 recovery because C141 was unwell at the time the “recovery sample” was taken (high CRP and hypoxia). They died a few days after.
adata = adata[(adata.obs['sample_id'] != 'C141_0903')].copy()
adata.n_obs

59553

In [8]:
# calculate days from first symptoms and positive swab
tmp = adata.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)
time_from_first_symptoms = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y')
time_from_positive_swab = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_from_first_symptoms = time_from_first_symptoms.dt.days
days_from_positive_swab = time_from_positive_swab.dt.days
days_from_first_symptoms.fillna(0, inplace = True)
days_from_positive_swab.fillna(0, inplace = True)

/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  return super().fillna(


In [9]:
min(days_from_first_symptoms)

-9.0

In [10]:
min(days_from_positive_swab)

0.0

In [11]:
days_from_infection = [max(a,b) for a,b in zip(days_from_first_symptoms, days_from_positive_swab)]

In [12]:
min(days_from_infection)

0.0

In [13]:
adata.obs['time_from_first_symptoms'] = days_from_first_symptoms
adata.obs['time_from_positive_swab'] = days_from_positive_swab
adata.obs['time_from_infection'] = days_from_infection

In [14]:
adata.write('h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_230622.h5ad', compression = 'gzip')

In [ ]:
# new session

In [1]:
import os
import tools
import numpy as np
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.22.4 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 pynndescent==0.5.2


In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
bdata = sc.read_h5ad('h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_230622.h5ad')
bdata

AnnData object with n_obs × n_vars = 59553 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [4]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [5]:
%%R
## Count cells in neighbourhoods
countCells_ <- function(x, samples, meta.data=NULL){

    # cast dplyr objects to data.frame
    if(!is.data.frame(meta.data) & !is.null(meta.data)){
        meta.data <- as.data.frame(meta.data)
    }

    if(length(samples) > 1 & !is.null(meta.data)){
        stop("Multiple sample columns provided, please specify a unique column name")
    } else if(is.null(meta.data) & length(samples) != ncol(x)){
        stop(paste0("Length of vector does not match dimensions of object. Length:",
                    length(samples), " Dimensions: ", ncol(x)))
    }

    # check the nhoods slot is populated
    if(ncol(nhoods(x)) == 1 & nrow(nhoods(x)) == 1){
        stop("No neighbourhoods found. Please run makeNhoods() first.")
    }

    message("Checking meta.data validity")
    if(!is.null(meta.data)){
        samp.ids <- unique(as.character(meta.data[, samples]))
    } else {
        samp.ids <- unique(as.character(samples))
    }

    num.hoods <- ncol(nhoods(x))

    ## Convert meta data to binary dummies in sparse matrix
    dummy.meta.data <- Matrix(data=0, nrow=nrow(meta.data), ncol = length(samp.ids), sparse = TRUE)
    colnames(dummy.meta.data) <- samp.ids
    rownames(dummy.meta.data) <- rownames(meta.data)
    for (s in seq_along(samp.ids)){
        i.s <- samp.ids[s]
        s.ixs <- which(meta.data[samples]==i.s)
        dummy.meta.data[s.ixs, as.character(i.s)] <- 1
    }

    message("Counting cells in neighbourhoods")
    count.matrix <- Matrix::t(nhoods(x)) %*% dummy.meta.data

    # add to the object
    rownames(count.matrix) <- c(1:num.hoods)
    nhoodCounts(x) <- count.matrix

    return(x)
}

### separate wave 1 and wave 2 because of large centre differences

### just wave 1

In [7]:
# store the neighborhood graph separately
adata_no_knnx = bdata[bdata.obs['centre'].isin(['NCL'])].copy()
# recompute the neighborhood
sc.pp.neighbors(adata_no_knnx, use_rep="X_scVI")
# Save the connectivity matrix
knn_adjacencyx = adata_no_knnx.obsp["connectivities"]
# pop the rest to make it work
adata_no_knnx.uns.pop("neighbors")
adata_no_knnx.obsm.pop("_scvi_extra_continuous")
adata_no_knnx.obsp = None

In [8]:
%%R -i adata_no_knnx -i knn_adjacencyx
save(countCells_, adata_no_knnx, knn_adjacencyx, file = 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave1_milo_prep_230622.RData')

### just wave 2

In [9]:
# Just keep patients that are positive in wave 2 but negative in wave 1
pt = ['C101', 'C108', 'C137', 'C138', 'C140', 'C141', 'C145', 'C146', 'C147', 'C168', 'C169', 'C170', 'C187', 'C190', 'C33',]
adata_no_knnx = bdata[bdata.obs['individual_id'].isin(pt)].copy()

#### find out which samples do not have complete data

In [10]:
# find out which samples do not have complete data
pd.crosstab(adata_no_knnx.obs['individual_id'], adata_no_knnx.obs['case_control'])

case_control,NEGATIVE,POSITIVE,RECOVERY
individual_id,,,
C33,0,861,0
C101,0,1187,36
C108,7,118,0
C137,497,1167,361
C138,49,655,0
C140,1323,5311,504
C141,0,533,0
C145,46,1252,0
C146,129,1376,252


#### for recovery vs neg

In [11]:
# remove incomplete data for for recovery vs neg
adata_no_knnx = adata_no_knnx[adata_no_knnx.obs['individual_id'].isin(['C137', 'C140', 'C146', 'C168', 'C169', 'C170', 'C187', 'C190'])].copy()
adata_no_knnx

AnnData object with n_obs × n_vars = 21740 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [12]:
# recompute the neighborhood
sc.pp.neighbors(adata_no_knnx, use_rep="X_scVI")
# Save the connectivity matrix
knn_adjacencyx = adata_no_knnx.obsp["connectivities"]
# pop the rest to make it work
adata_no_knnx.uns.pop("neighbors")
adata_no_knnx.obsm.pop("_scvi_extra_continuous")
adata_no_knnx.obsp = None

In [13]:
%%R -i adata_no_knnx -i knn_adjacencyx
save(countCells_, adata_no_knnx, knn_adjacencyx, file = 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo_prep_230622.RData')

#### for positive vs neg

In [14]:
adata_no_knnx = bdata[bdata.obs['individual_id'].isin(pt)].copy()

In [15]:
# remove incomplete data
adata_no_knnx = adata_no_knnx[adata_no_knnx.obs['individual_id'].isin(['C108', 'C137', 'C138', 'C140', 'C145', 'C146', 'C147', 'C168', 'C169', 'C170', 'C187', 'C190',])].copy()
adata_no_knnx

AnnData object with n_obs × n_vars = 26458 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [16]:
# recompute the neighborhood
sc.pp.neighbors(adata_no_knnx, use_rep="X_scVI")
# Save the connectivity matrix
knn_adjacencyx = adata_no_knnx.obsp["connectivities"]
# pop the rest to make it work
adata_no_knnx.uns.pop("neighbors")
adata_no_knnx.obsm.pop("_scvi_extra_continuous")
adata_no_knnx.obsp = None

In [17]:
%%R -i adata_no_knnx -i knn_adjacencyx
save(countCells_, adata_no_knnx, knn_adjacencyx, file = 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_pos_vs_neg_milo_prep_230622.RData')